In [3]:
import pandas as pd
import json
import numpy as np
from collections import OrderedDict

In [73]:
fulldf = pd.read_csv('/Users/willb/Documents/GitHub/efficient-learning-khan/code/eff data/efflearnkhan_14.csv', sep='\t')

data = []
fulldf
for row in range(len(fulldf)):
    data.append((fulldf.loc[row, 'uniqueid'], fulldf.loc[row, 'datastring']))

d = json.loads(data[0][1])

e = d['data']
    
for trial in e:
    # if the json block contains answers, put the text in subas
    if 'responses' in trial['trialdata']:
        subas.append([i.replace("&#8217;","\'") for i in OrderedDict(json.loads(trial['trialdata']['responses'])).values()])
    
    # if the json block contains questions, put the text in subqs
    elif isinstance(trial['trialdata'], type(list)):
        subqs.append([qset['prompt'].replace("&#8217;","\'") for qset in trial['trialdata']])

In [9]:
data_series = pd.Series(fulldf['datastring'])

In [75]:
# qdict = {}  # maps question => [answer, question ID, video number]

# infile = open("../data analysis/astronomyquestions.tsv", "r")

# parts = []
# for line in infile.read().split("\n"):
#     parts.append(line.split("\t"))

# for line in parts:
#     qdict[line[2]] = [line[3], line[0], line[1]]
    
# for q in qdict:
#     print(qdict[q][0])
    
dict1 = {}
dict1["one"] = 1
dict1["two"] = 2
dict1["three"] =  "stuff1"
dict2 = {}
dict2["one"] = 1
dict2["two"] = 2
dict2["three"] = "stuff2"
dict_list = [dict1, dict2]
df1 = pd.DataFrame(dict_list)

df1



,one,three,two
0,1,stuff1,2
1,1,stuff2,2


In [11]:
def grade(output, questiondict, qsets, asets):
    outfile = open(output, "w")
    for set in range(3):    # for each of the 3 sets
        for question in range(13):
            if qsets[set][question] in questiondict:    # check if we have the question
                outfile.write(str(set) + "\t" + str(questiondict[qsets[set][question]][0]) + "\t")
                if asets[set][question] == questiondict[qsets[set][question]][1]:   # check if the answer is right
                    outfile.write("1")
                else:
                    outfile.write("0")
                outfile.write("\n")
            else:
                outfile.write("error" + "\n")
    outfile.close()

In [12]:
qdict = readquestions('../data analysis/testvideoquestions')

NameError: name 'readquestions' is not defined

In [17]:
for n, sub in enumerate(data_series):
    subqs = []
    subas = []
    for trial in sub['data']:
        
        if 'responses' in trial['trialdata']:
            subas.append([i.replace("&#8217;","\'") for i in OrderedDict(json.loads(trial['trialdata']['responses'])).values()])
               
        elif type(trial['trialdata']) is list:
            subqs.append([qset['prompt'].replace("&#8217;","\'") for qset in trial['trialdata']])
            
    if n == 1:
        print(subqs[0])
        print('\n\n')
        print(subas[0])
        break
    if n == 3:
        subqs = subqs[1:]
    if len(subqs) > 3:
        print('problem with sub ' + str(n) + '. q length of ' + str(len(subqs)))
    if len(subas) > 3:
        print('problem with sub ' + str(n) + '. a length of ' + str(len(subas)))
    #grade("participant" + str(n + 1) + "data.tsv", qdict, subqs, subas)  

        
    print(" I did one")

13
 I did one
[]
